In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_regression


In [37]:
df = pd.read_csv('../../data/processed/combined_data_v4_nearby_road_1km.csv')

In [38]:
df.head()

,address_subdivision,address_locality,address_line_2,h_id,category_name,type,price_per_m2,land_area,price,longitude,...,f_tertiary,f_track,f_trunk,f_trunk_link,f_unclassified,f_unused,mean,median,max,min
0,Phnom Penh,Meanchey,Chak Angrae Leu,8865846ac7fffff,Flat,residential,1000.000000,420.0,420000.0,104.913586,...,1,0,1,1,0,0,2702.583583,2343.750000,6465.517241,110.000000
1,Phnom Penh,Por Sen Chey,Kakap,8865846f01fffff,Apartment,residential,804.597701,435.0,350000.0,104.842882,...,1,0,1,0,0,0,1235.634744,1235.633333,1666.670000,804.597701
2,Phnom Penh,Sen Sok,Tuek Thla,8865846f69fffff,Apartment,residential,1971.326165,837.0,1650000.0,104.871316,...,1,1,1,0,0,0,1779.287046,1764.710000,1971.330000,1600.000000
3,Phnom Penh,Chamkarmon,BKK 3,8865846ac7fffff,Apartment,residential,6021.505376,465.0,2800000.0,104.913586,...,1,0,1,1,0,0,2702.583583,2343.750000,6465.517241,110.000000
4,Phnom Penh,Dangkao,Dangkao,8865846183fffff,House,residential,927.835052,485.0,450000.0,104.856173,...,1,1,0,0,0,0,1408.401654,1379.598571,2941.176471,120.000000


In [39]:
# df = df[df['price_per_m2'] >= 100].copy()

In [40]:
# df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna(0)

In [41]:
# # Calculate mean, max, min of 'price_per_m2' grouped by 'h_id'
# price_stats = df.groupby('address_line_2')['price_per_m2'].agg(['mean','median', 'max', 'min']).reset_index()

# # Merge these stats back to the original dataframe on 'h_id'
# df = df.merge(price_stats, on='address_line_2', how='left', suffixes=('', '_price_per_m2'))

# # The columns 'mean', 'max', 'min' are now added to df

In [42]:
# df.drop(columns=['price_display'], inplace=True)

In [43]:
df.drop(columns=['price'], inplace=True)

In [44]:
df.dropna(inplace=True)

In [45]:
y = df[['price_per_m2']]
X = df.drop(columns=['price_per_m2'])

In [46]:
X.describe()

,land_area,longitude,latitude,population,nearest_cafe,n_cafe_in_1km,n_cafe_in_1km_to_2km,n_cafe_in_2km_to_3km,n_cafe_in_3km_to_5km,nearest_gas_station,...,f_tertiary,f_track,f_trunk,f_trunk_link,f_unclassified,f_unused,mean,median,max,min
count,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,...,8180.000000,8180.000000,8180.000000,8180.000000,8180.000000,8180.0,8180.000000,8180.000000,8180.000000,8180.000000
mean,278.333649,104.897303,11.555653,8845.751589,1.777384,4.170293,13.592787,20.064181,49.774817,1.031785,...,0.933374,0.660391,0.464181,0.307457,0.340831,0.0,1886.490778,1759.738841,3704.108828,841.372922
std,2149.972453,0.045727,0.047513,8774.838456,2.743321,5.359708,14.124205,18.213724,34.188368,1.661093,...,0.249388,0.473605,0.498746,0.461469,0.474018,0.0,876.571058,836.608137,2410.555936,847.102262
min,1.000000,104.740403,11.427535,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,100.000000,100.000000,100.000000,100.000000
25%,58.000000,104.872521,11.527183,1971.000000,0.000000,0.000000,2.000000,3.000000,15.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1332.999731,1269.165000,1666.670000,210.000000
50%,84.000000,104.906379,11.550000,5039.000000,0.000000,2.000000,8.000000,13.000000,56.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,1961.769279,1819.460899,3333.333333,535.710000
75%,112.000000,104.926502,11.582277,15603.000000,3.000000,7.000000,24.000000,36.000000,80.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,2430.555051,2192.599286,5902.777778,1234.375000
max,104635.000000,105.003399,11.721649,39519.000000,15.000000,29.000000,61.000000,72.000000,125.000000,13.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,6435.640170,6435.640000,8870.967742,6435.640000


In [47]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.shape, X_test.shape

In [48]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns


In [49]:
cat_cols

Index(['address_subdivision', 'address_locality', 'address_line_2', 'h_id',
       'category_name', 'type'],
      dtype='object')

In [50]:
# Temporarily encode categorical columns for feature selection
X_temp = X.copy()
if len(cat_cols) > 0:
    encoder = OrdinalEncoder()
    X_temp[cat_cols] = encoder.fit_transform(X_temp[cat_cols])


In [51]:
# Feature selection 
selector = SelectKBest(mutual_info_regression, k=30)
selector.fit(X_temp, y)
selected_features = X.columns[selector.get_support()]


d:\anaconda\envs\env_v3.10\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
# Subset original data to selected features
X_selected = X[selected_features]

In [53]:
X_selected

,address_locality,address_line_2,h_id,category_name,longitude,latitude,population,n_cafe_in_1km_to_2km,n_cafe_in_2km_to_3km,n_cafe_in_3km_to_5km,...,n_university_in_3km_to_5km,n_seven_eleven_in_3km_to_5km,n_super_market_in_3km_to_5km,n_borey_in_3km_to_5km,n_bank_in_3km_to_5km,n_atm_in_3km_to_5km,mean,median,max,min
0,Meanchey,Chak Angrae Leu,8865846ac7fffff,Flat,104.913586,11.544500,23239.0,36,37,85,...,62,18,35,18,7,83,2702.583583,2343.750000,6465.517241,110.000000
1,Por Sen Chey,Kakap,8865846f01fffff,Apartment,104.842882,11.563386,6724.0,9,4,24,...,19,5,10,17,2,5,1235.634744,1235.633333,1666.670000,804.597701
2,Sen Sok,Tuek Thla,8865846f69fffff,Apartment,104.871316,11.557682,12095.0,7,32,71,...,41,13,28,31,10,38,1779.287046,1764.710000,1971.330000,1600.000000
3,Chamkarmon,BKK 3,8865846ac7fffff,Apartment,104.913586,11.544500,23239.0,36,37,85,...,62,18,35,18,7,83,2702.583583,2343.750000,6465.517241,110.000000
4,Dangkao,Dangkao,8865846183fffff,House,104.856173,11.500658,2310.0,2,0,15,...,2,6,3,16,0,0,1408.401654,1379.598571,2941.176471,120.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8192,Chbar Ampov,Nirouth,8865846a0bfffff,Shophouse,104.959508,11.535064,1726.0,3,10,51,...,11,12,32,7,7,42,3388.889947,3388.890000,3388.890000,3388.888889
8193,Sen Sok,Phnom Penh Thmey,8865846f25fffff,Land/Development,104.854256,11.592307,2842.0,4,9,41,...,41,7,15,13,2,7,460.339997,460.340000,460.340000,460.339943
8194,Sen Sok,Phnom Penh Thmey,886584688bfffff,Land/Development,104.877009,11.603910,6031.0,8,7,55,...,63,12,21,16,2,23,1872.659961,1872.660000,1872.660000,1872.659176
8195,Sen Sok,Krang Thnong,8865846f27fffff,House,104.844053,11.590668,524.0,4,5,27,...,20,6,8,12,2,5,2000.000000,2000.000000,2000.000000,2000.000000


In [54]:
# One-hot encode only the selected categorical features
selected_cat_cols = [col for col in selected_features if col in cat_cols]
X_final = pd.get_dummies(X_selected, columns=selected_cat_cols, drop_first=True)

# Only convert one-hot columns to int
one_hot_cols = [col for col in X_final.columns if any(cat in col for cat in selected_cat_cols)]
X_final[one_hot_cols] = X_final[one_hot_cols].astype(int)

In [55]:
X_final

,longitude,latitude,population,n_cafe_in_1km_to_2km,n_cafe_in_2km_to_3km,n_cafe_in_3km_to_5km,n_gas_station_in_2km_to_3km,n_gas_station_in_3km_to_5km,n_hospital_in_3km_to_5km,n_hotel_in_3km_to_5km,...,category_name_Shophouse,category_name_Studio,category_name_Terrace,category_name_Twin Villa,category_name_Unit,category_name_Villa,category_name_Warehouse,category_name_condo,category_name_project,category_name_residential
0,104.913586,11.544500,23239.0,36,37,85,35,76,22,47,...,0,0,0,0,0,0,0,0,0,0
1,104.842882,11.563386,6724.0,9,4,24,15,39,5,9,...,0,0,0,0,0,0,0,0,0,0
2,104.871316,11.557682,12095.0,7,32,71,31,95,45,11,...,0,0,0,0,0,0,0,0,0,0
3,104.913586,11.544500,23239.0,36,37,85,35,76,22,47,...,0,0,0,0,0,0,0,0,0,0
4,104.856173,11.500658,2310.0,2,0,15,2,28,1,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8192,104.959508,11.535064,1726.0,3,10,51,4,27,28,37,...,1,0,0,0,0,0,0,0,0,0
8193,104.854256,11.592307,2842.0,4,9,41,15,53,6,8,...,0,0,0,0,0,0,0,0,0,0
8194,104.877009,11.603910,6031.0,8,7,55,14,67,11,8,...,0,0,0,0,0,0,0,0,0,0
8195,104.844053,11.590668,524.0,4,5,27,14,45,1,8,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df_KBest = pd.concat([X_final, y], axis=1)


In [57]:
df_KBest.head()

,longitude,latitude,population,n_cafe_in_1km_to_2km,n_cafe_in_2km_to_3km,n_cafe_in_3km_to_5km,n_gas_station_in_2km_to_3km,n_gas_station_in_3km_to_5km,n_hospital_in_3km_to_5km,n_hotel_in_3km_to_5km,...,category_name_Studio,category_name_Terrace,category_name_Twin Villa,category_name_Unit,category_name_Villa,category_name_Warehouse,category_name_condo,category_name_project,category_name_residential,price_per_m2
0,104.913586,11.544500,23239.0,36,37,85,35,76,22,47,...,0,0,0,0,0,0,0,0,0,1000.000000
1,104.842882,11.563386,6724.0,9,4,24,15,39,5,9,...,0,0,0,0,0,0,0,0,0,804.597701
2,104.871316,11.557682,12095.0,7,32,71,31,95,45,11,...,0,0,0,0,0,0,0,0,0,1971.326165
3,104.913586,11.544500,23239.0,36,37,85,35,76,22,47,...,0,0,0,0,0,0,0,0,0,6021.505376
4,104.856173,11.500658,2310.0,2,0,15,2,28,1,3,...,0,0,0,0,0,0,0,0,0,927.835052


In [59]:
df_KBest.to_csv('../../data/preprocessed/combined_data_preprocessed_Kbest_v1.csv', index=False)